# Answering Business Questions Using SQL

## 1. Introduction and Schema Diagram

In this project, we're going to practice using our SQL skills to answer business questions.

We'll use the Chinook database. The Chinook database is provided as a SQLite database file called **chinook.db**  
A copy of the database schema is below.

![title](chinook.png)

We'll use the following code to connect our Jupyter Notebook to our database file:

In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

## 2. Overview of the Data

Let's start by getting familiar with our data. We can query the database to get a list of all tables and views in our database. 

To run SQL queries in Jupyter Notebook, we need to add '**%%sql**' to the start of our query. 

So, we'll use this code to view all tables and views in our db:

In [4]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## 3. Analysis

### 3.1. Selecting New Albums to Purchase

The Chinook record store has just signed a deal with a new record label, and we've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

| Artist Name          | Genre   |
|----------------------|---------|
| Regal                | Hip-Hop |
| Red Tone             | Punk    |
| Meteor and the Girls | Pop     |
| Slim Jim Bites       | Blues   |

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

We'll need to write a query to find out which genres sell the most tracks in the USA.

In [25]:
%%sql
WITH usa_tracks_sold AS (
    SELECT il.* FROM invoice_line il
    LEFT JOIN invoice i on il.invoice_id = i.invoice_id
    LEFT JOIN customer c on i.customer_id = c.customer_id
    WHERE c.country = "USA"
    )

SELECT 
    g.name AS genre,
    SUM(il.quantity) AS quantity_USA,
    ROUND(CAST(SUM(il.quantity) AS FLOAT)*100 / (SELECT SUM(quantity) FROM usa_tracks_sold), 2) AS percentage
FROM invoice_line il
LEFT JOIN track t ON il.track_id = t.track_id
LEFT JOIN genre g ON t.genre_id = g.genre_id
LEFT JOIN invoice i ON i.invoice_id = il.invoice_id
LEFT JOIN customer c ON c.customer_id = i.customer_id
WHERE c.country = 'USA'
GROUP BY genre
ORDER BY quantity_USA DESC;


 * sqlite:///chinook.db
Done.


genre,quantity_USA,percentage
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Pop,22,2.09
Latin,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


Based on our findings regarding performance of genres in the USA, we recommend the new albums by the following artists, according to their national sales performance:

- Red Tone (Punk)
- Slim Jim Bites (Blues)
- Meteor and the Girls (Pop)

Rock is the best-selling genre in the USA by far. Unfortunately, we don't have a Rock album in our list.
Therefore, we strongly suggest management to be on the lookout for albums in the Rock genre, which accounts for 53% of sales in the USA!

### 3.2. Analyzing Employee Sales Performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [44]:
%%sql
WITH last_invoice_date AS (
    SELECT MAX(invoice_date) FROM invoice
    )

SELECT 
    e.employee_id,
    e.first_name || " " || e.last_name AS employee_name,
    e.title AS employee_title,
    e.hire_date,
    ROUND(SUM(i.total), 0) AS sales,
    JulianDay((SELECT * FROM last_invoice_date)) - JulianDay(e.hire_date) AS days_employed,
    ROUND(SUM(i.total) / ( JulianDay((SELECT * FROM last_invoice_date)) - JulianDay(e.hire_date) ),2) AS 'performance'
FROM invoice i
LEFT JOIN customer c ON c.customer_id = i.customer_id
LEFT JOIN employee e ON e.employee_id = c.support_rep_id
GROUP BY e.employee_id
ORDER BY sales DESC;
    
    

 * sqlite:///chinook.db
Done.


employee_id,employee_name,employee_title,hire_date,sales,days_employed,performance
3,Jane Peacock,Sales Support Agent,2017-04-01 00:00:00,1732.0,1369.0,1.26
4,Margaret Park,Sales Support Agent,2017-05-03 00:00:00,1584.0,1337.0,1.18
5,Steve Johnson,Sales Support Agent,2017-10-17 00:00:00,1394.0,1170.0,1.19


We may say that Jane is the best performing Sales Support Agent by looking at the **sales** column, **but that'd be an immature statement**. 

If we look at the **hire_date** column, we can see that Jane has worked for Chinook 1 month longer than Margaret, and 5.5 months longer than Steve. 

That's why I calculated the days_employed value as:

**days_employed = hire_date - date of last invoice in the invoice table**

Then:

**performance = sales / days_employed**

Then... Well...  
Jane still happens to be the best performing Sales Support Agent. But hey! At least we can sleep comfortably now :)

On average, **Jane brings 0.08 more sales value every day**, compared to other two Sales Support Agents, Margaret and Steve.


Also, we can see that performance of Steve and Margaret are almost the same, altough Margaret had 1584 where Steve had 1394 in sales. By going the extra step and calculating the **performance** metric, **we didn't falsely say "Steve is the worst performer!"**. Indeed, he performs slightly (but slightly) better.

### 3.3. Analyzing Sales by Country

Our next task is to analyze the sales data for customers from each different country. We have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, we have been directed to calculate data, for each country, on the:

- total number of customers
- total value of sales
- average value of sales per customer
- average order value

Because there are a number of countries with only one customer, we should group these customers as "Other", and we should force the ordering of "Other" to last in our analysis.

In [79]:
%%sql
WITH 

customers_per_country AS(
    SELECT
        c.country as country,
        COUNT(c.customer_id) as customers,
    
        CASE 
            WHEN COUNT(c.customer_id) = 1 THEN 'Other'
            ELSE c.country
        END AS country_helper,
        
        CASE
            WHEN COUNT(c.customer_id) = 1 THEN 1
            ELSE 0
        END AS rank_helper
    FROM customer c
    GROUP BY country
),

customer_adjusted AS(
    SELECT 
        c.*,
        cpc.country_helper,
        cpc.rank_helper
    FROM customer c
    LEFT JOIN customers_per_country cpc ON cpc.country = c.country
)


SELECT
    cpc.country_helper                                           AS country,
    COUNT(DISTINCT c.customer_id)                                AS customers,
    ROUND(    SUM(i.total)                                   ,2) AS  total_sales,
    ROUND(    SUM(i.total) / COUNT(distinct i.invoice_id)    ,2) AS avg_order_value,
    ROUND(    SUM(i.total) / COUNT(distinct c.customer_id)   ,2) AS avg_sales_per_customer
    
FROM invoice i

LEFT JOIN customer c ON c.customer_id = i.customer_id
LEFT JOIN customers_per_country cpc ON cpc.country = c.country

GROUP BY cpc.country_helper
ORDER BY rank_helper, total_sales DESC;

 * sqlite:///chinook.db
Done.


country,customers,total_sales,avg_order_value,avg_sales_per_customer
USA,13,1040.49,7.94,80.04
Canada,8,535.59,7.05,66.95
Brazil,5,427.68,7.01,85.54
France,5,389.07,7.78,77.81
Germany,4,334.62,8.16,83.66
Czech Republic,2,273.24,9.11,136.62
United Kingdom,3,245.52,8.77,81.84
Portugal,2,185.13,6.38,92.57
India,2,183.15,8.72,91.58
Other,15,1094.94,7.45,73.0


### 3.4. Albums vs Individual Tracks

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

- purchase a whole album
- purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

It is very common when you are performing an analysis to have 'edge cases' which prevent you from getting a 100% accurate answer to your question. In this instance, we have two edge cases to consider:

- Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.
- Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.

In the first case, since our analysis is concerned with maximizing revenue we can safely ignore albums consisting of only a few tracks. The company has previously done analysis to confirm that the second case does not happen often, so we can ignore this case also.

In order to answer the question, we're going to have to identify whether each invoice has all the tracks from an album. We can do this by getting the list of tracks from an invoice and comparing it to the list of tracks from an album. We can find the album to compare the purchase to by looking up the album that one of the purchased tracks belongs to. It doesn't matter which track we pick, since if it's an album purchase, that album will be the same for all tracks.

In [86]:
%%sql
WITH 
invoice_first_track AS (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
)


SELECT 
    album_or_not,
    COUNT(invoice_id) AS invoices,
    ROUND( CAST(COUNT(invoice_id) AS FLOAT)*100 / (SELECT COUNT(*) FROM invoice)  ,2) AS percentage
FROM (
    SELECT
        ifs.*,
        CASE
            WHEN
                (SELECT t.track_id FROM track t
                 WHERE t.album_id = (SELECT t2.album_id FROM track t2
                                     WHERE t2.track_id = ifs.first_track_id
                                    )
                EXCEPT
                 
                SELECT il2.track_id FROM invoice_line il2
                WHERE il2.invoice_id = ifs.invoice_id
                ) IS NULL
            
            AND
                (SELECT il2.track_id FROM invoice_line il2
                 WHERE il2.invoice_id = ifs.invoice_id
                 
                 EXCEPT
                 
                 SELECT t.track_id FROM track t
                 WHERE t.album_id = (SELECT t2.album_id FROM track t2
                                     WHERE t2.track_id = ifs.first_track_id
                                    )
                ) IS NULL
            
            THEN "yes"
            ELSE "no"
        END AS "album_or_not"
    
    FROM invoice_first_track AS ifs
    )
GROUP BY album_or_not;



 * sqlite:///chinook.db
Done.


album_or_not,invoices,percentage
no,500,81.43
yes,114,18.57


Album sales account for 18.57% of total sales.  
About 1 in every 5 purchase is an Album.

I think this is a quite significant share in all sales.  
It shows that there are customers who value albums as a whole, rather than individual tracks in them.

Therefore I'd advice against a strategy of providing only 'select tracks'.  
It may result in significant loss in revenue.

## 4. Extra Questions

### Which artist is used in the most playlists?

In [89]:
%%sql
SELECT 
    a.artist_id,
    a.name artist_name,
    COUNT(DISTINCT p.playlist_id) as playlist_appearance
FROM playlist p 
LEFT JOIN playlist_track pt ON pt.playlist_id = p.playlist_id
LEFT JOIN track t ON t.track_id = pt.track_id
LEFT JOIN album al ON al.album_id = t.album_id
LEFT JOIN artist a ON a.artist_id = al.artist_id
GROUP BY a.artist_id
ORDER BY playlist_appearance DESC
LIMIT 5;


 * sqlite:///chinook.db
Done.


artist_id,artist_name,playlist_appearance
226,Eugene Ormandy,7
248,Berliner Philharmoniker & Herbert Von Karajan,6
247,The King's Singers,6
214,Academy of St. Martin in the Fields & Sir Neville Marriner,6
208,English Concert & Trevor Pinnock,6


### How many tracks have been purchased vs not purchased?

In [99]:
%%sql

WITH
track_purchase AS (
    SELECT
        t.track_id,
        CASE
            WHEN t.track_id IN (SELECT il.track_id FROM invoice_line il) THEN "Yes"
            ELSE "No"
        END AS purchased
    FROM track t
)

SELECT 
    purchased,
    COUNT(purchased) AS count,
    ROUND( CAST(COUNT(purchased) AS FLOAT)*100 / (SELECT COUNT(*) FROM track)  ,2) AS percentage
    
FROM track_purchase
GROUP BY purchased
ORDER BY purchased DESC;

 * sqlite:///chinook.db
Done.


purchased,count,percentage
Yes,1806,51.56
No,1697,48.44


### Do protected vs non-protected media types have an effect on popularity?

In [113]:
%%sql
-- PROTECTED VS NOT PROTECTED SHARE IN ALL TRACKS

WITH protected_or_not AS(
    SELECT t.track_id,
    CASE
        WHEN mt.name LIKE "%Protected%" THEN "Protected"
        ELSE "Not Protected"
    END AS type
    FROM track t
    LEFT JOIN media_type mt ON mt.media_type_id = t.media_type_id
)

SELECT 
    pon.type AS all_tracks,
    COUNT(pon.type) AS count,
    ROUND(  CAST(COUNT(pon.type) AS FLOAT)*100 / (SELECT COUNT(*) FROM protected_or_not)  ,2) AS percentage
FROM protected_or_not pon
GROUP BY all_tracks;

 * sqlite:///chinook.db
Done.


all_tracks,count,percentage
Not Protected,3052,87.13
Protected,451,12.87


In [114]:
%%sql
-- PROTECTED VS NOT PROTECTED MEDIA TYPES IN SALES

WITH protected_or_not AS(
    SELECT t.track_id,
    CASE
        WHEN mt.name LIKE "%Protected%" THEN "Protected"
        ELSE "Not Protected"
    END AS type
    FROM track t
    LEFT JOIN media_type mt ON mt.media_type_id = t.media_type_id
)

SELECT 
    pon.type AS sold_tracks,
    COUNT(pon.type) AS count,
    ROUND(  CAST(COUNT(pon.type) AS FLOAT)*100 / (SELECT COUNT(*) FROM invoice_line)  ,2) AS percentage
FROM invoice_line il
LEFT JOIN protected_or_not pon ON pon.track_id = il.track_id
GROUP BY sold_tracks;    

 * sqlite:///chinook.db
Done.


sold_tracks,count,percentage
Not Protected,4315,90.71
Protected,442,9.29


**'Not Protected'** media type has a 87% share in all tracks, but has 90% share in total sales.

Results suggest that **'Not Protected'** media files are slightly more popular on the market.